# View Color light-Curves and Histograms for repeatability

- author Sylvie Dagoret-Campagne
- creation date 2024-09-07
- last update 2024-09-07

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import SigmaClip


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

import scipy
from scipy.optimize import curve_fit

In [ ]:
#inputfilename = "sources_objectTable-t3864-multibands-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"

## The two lists:
theta_sep = 300.
time_sep = 1800.
NMAX = 50
CBAND = "G-R"
CBANDtag = "G_R"
BAND1 = "G"
BAND2 = "R"
path_colorlc = f"data_coloredlightcurves{CBANDtag}_thmax_{theta_sep:.0f}_tmax_{time_sep:.0f}"

summary_file = f"photompairsummary{CBANDtag}_thmax_{theta_sep:.0f}_tmax_{time_sep:.0f}.csv"
fntag = f"^color_lightcurve{CBANDtag}_(.*)_thmax_{theta_sep:.0f}_tmax_{time_sep:.0f}.csv$"

summary_fullfilename= os.path.join(path_colorlc,summary_file) 

dfc = pd.read_csv(summary_fullfilename,index_col=0)

In [ ]:
dfc

In [ ]:
len(dfc.idx_pair.unique())

In [ ]:
list_of_files = os.listdir(path_colorlc) 

In [ ]:
all_pairnum = []
for idx,file in enumerate(list_of_files):
    coll = re.findall(fntag,file)
    if len(coll)>0:
        all_pairnum.append(int(coll[0]))
        
all_pairnum = sorted(all_pairnum)  

In [ ]:
print(all_pairnum) 

In [ ]:
all_df = []
for idx,idx_pair in enumerate(all_pairnum):
    idx_pair_str = str(idx_pair).zfill(4)
    filename =  f"color_lightcurve{CBANDtag}_{idx_pair_str}_thmax_{theta_sep:.0f}_tmax_{time_sep:.0f}.csv"
    fullfilename = os.path.join(path_colorlc,filename)
    df = pd.read_csv(fullfilename,index_col=0)
    all_df.append(df)
    

In [ ]:
df = all_df[0]

### Definitions

            # first indec band, second index star
            # Star 1 in band 1
            psfMag11 =  row_pair_b1["psfMag_x"]
            
            if FLAG_AP_FLUX_CAL:
                apMag11 =  row_pair_b1["apFlux_35_0_calMag_x"]
            else:
                apMag11 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_x"])
 
            # Star 2 in band 1
            psfMag12 =  row_pair_b1["psfMag_y"]


            if FLAG_AP_FLUX_CAL:
                apMag12 =  row_pair_b1["apFlux_35_0_calMag_y"]
  
            else:
                apMag12 =  -2.5*np.log10(row_pair_b1["apFlux_35_0_instFlux_y"])
 
            # Star 1 in band 2
            psfMag21 =  row_pair_b2["psfMag_x"]

            if FLAG_AP_FLUX_CAL:
                apMag21 =  row_pair_b2["apFlux_35_0_calMag_x"]
            else:
                apMag21 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_x"])

            
            # Star 2 in band 2
            psfMag22 =  row_pair_b2["psfMag_y"]

            
            if FLAG_AP_FLUX_CAL:
                apMag22 =  row_pair_b2["apFlux_35_0_calMag_y"]
            else:
                apMag22 =  -2.5*np.log10(row_pair_b2["apFlux_35_0_instFlux_y"])

        

In [ ]:
df.head()

        # band 1 : mag_star1-mag_star2 
        df_col_b1b2["deltapsfmag_s12b1"] =    (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag12"])
        df_col_b1b2["deltapsfmagErr_s12b1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr12"]**2)

        df_col_b1b2["deltaapmag_s12b1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag12"])
        df_col_b1b2["deltaapmagErr_s12b1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr12"]**2)
    
        # band 2 : mag_star1-mag_star2 
        df_col_b1b2["deltapsfmag_s12b2"] = (df_col_b1b2["psfMag21"] - df_col_b1b2["psfMag22"])
        df_col_b1b2["deltapsfmagErr_s12b2"] = np.sqrt(df_col_b1b2["psfMagErr21"]**2 + df_col_b1b2["psfMagErr22"]**2)
        df_col_b1b2["deltaapmag_s12b2"] = (df_col_b1b2["apMag21"] - df_col_b1b2["apMag22"])
        df_col_b1b2["deltaapmagErr_s12b2"] = np.sqrt(df_col_b1b2["apMagErr21"]**2 + df_col_b1b2["apMagErr22"]**2)

    
        # Star 1 : mag_band1-mag_band2 
        df_col_b1b2["psfcol12_s1"] = (df_col_b1b2["psfMag11"] - df_col_b1b2["psfMag21"])
        df_col_b1b2["psfcol12Err_s1"] = np.sqrt(df_col_b1b2["psfMagErr11"]**2 + df_col_b1b2["psfMagErr21"]**2)
        df_col_b1b2["apcol12_s1"] = (df_col_b1b2["apMag11"] - df_col_b1b2["apMag21"])
        df_col_b1b2["apcol12Err_s1"] = np.sqrt(df_col_b1b2["apMagErr11"]**2 + df_col_b1b2["apMagErr21"]**2)

    
       # Star 2 : mag_band1-mag_band2 
       df_col_b1b2["psfcol12_s2"] = (df_col_b1b2["psfMag12"] - df_col_b1b2["psfMag22"])
       df_col_b1b2["psfcol12Err_s2"] = np.sqrt(df_col_b1b2["psfMagErr12"]**2 + df_col_b1b2["psfMagErr22"]**2)
       df_col_b1b2["apcol12_s2"] = (df_col_b1b2["apMag12"] - df_col_b1b2["apMag22"])
       df_col_b1b2["apcol12Err_s2"] = np.sqrt(df_col_b1b2["apMagErr12"]**2 + df_col_b1b2["apMagErr22"]**2)

    
       # relative color star1 - star2
       df_col_b1b2["psfcol12_s12"] = np.abs(df_col_b1b2["psfcol12_s1"] - df_col_b1b2["psfcol12_s2"])
       df_col_b1b2["psfcol12Err_s12"] = np.sqrt(df_col_b1b2["psfcol12Err_s1"]**2 + df_col_b1b2["psfcol12Err_s2"]**2)
       df_col_b1b2["apcol12_s12"] = np.abs(df_col_b1b2["apcol12_s1"] - df_col_b1b2["apcol12_s2"])
       df_col_b1b2["apcol12Err_s12"] = np.sqrt(df_col_b1b2["apcol12Err_s1"]**2 + df_col_b1b2["apcol12Err_s2"]**2)


       # Maximum magnitude in band b1 and band b2
       df_col_b1b2["psfmagmax_s12b1"] = np.fmax(df_col_b1b2["psfMag11"],df_col_b1b2["psfMag12"])
       df_col_b1b2["psfmagmax_s12b2"] = np.fmax(df_col_b1b2["psfMag21"],df_col_b1b2["psfMag22"])
       df_col_b1b2["apmagmax_s12b1"] = np.fmax(df_col_b1b2["apMag11"],df_col_b1b2["apMag12"])
       df_col_b1b2["apmagmax_s12b2"] = np.fmax(df_col_b1b2["apMag21"],df_col_b1b2["apMag22"])
    

In [ ]:
residuals = []
avgpsfmag = []

FLAG_PSF = False

In [ ]:
       # relative color star1 - star2
       #df_col_b1b2["psfcol12_s12"] = np.abs(df_col_b1b2["psfcol12_s1"] - df_col_b1b2["psfcol12_s2"])
       #df_col_b1b2["psfcol12Err_s12"] = np.sqrt(df_col_b1b2["psfcol12Err_s1"]**2 + df_col_b1b2["psfcol12Err_s2"]**2)
       #df_col_b1b2["apcol12_s12"] = np.abs(df_col_b1b2["apcol12_s1"] - df_col_b1b2["apcol12_s2"])
       #df_col_b1b2["apcol12Err_s12"] = np.sqrt(df_col_b1b2["apcol12Err_s1"]**2 + df_col_b1b2["apcol12Err_s2"]**2)

In [ ]:
import matplotlib.dates as mdates
month_locator = mdates.MonthLocator(interval=1)
day_locator = mdates.DayLocator(interval=1)
year_month_formatter = mdates.DateFormatter("%Y-%m") 


# loop on individual objects
for idx,df in enumerate(all_df):
    pair_num = idx
          
   
    X = pd.to_datetime(df['time1'].values)  
    
    # convert in mmag
    if FLAG_PSF:    
        Y = df['psfcol12_s12'].values*1000.
        Yerr = df['psfcol12Err_s12'].values*1000.        
    else:  
        Y = df['apcol12_s12'].values*1000.
        Yerr = df['apcol12Err_s12'].values*1000.
                 
        
    N = len(Y)
    
    if N<NMAX:
        continue
        
    clippedMeanMagArray = scipy.stats.sigmaclip(Y, low=3.0, high=3.0).clipped
    clippedMeanMag = np.mean(clippedMeanMagArray)
    clippedMeanMagErr = np.std(clippedMeanMagArray)/np.sqrt(len(clippedMeanMagArray))
    clippeErrorMagArray = scipy.stats.sigmaclip(Y-clippedMeanMag,low=3.0, high=3.0).clipped
    clippeErrorMagSigma = (np.sqrt(np.sum(clippeErrorMagArray**2))/np.sqrt(len(clippeErrorMagArray)))

 
    ## Plot the color light curve
    
    fig,ax = plt.subplots(1,1,figsize=(12,4))
    
    magmax = df["apmagmax_s12b1"].mean()
  
    
    avgpsfmag.append(magmax)
    residuals.append(clippeErrorMagArray)
              
    label = f" band {CBAND} : magmax({BAND1}) = {magmax:.1f} mag , rms = {clippeErrorMagSigma:.0f} mmag, N= {N}"
    ax.errorbar(X,(Y-clippedMeanMag),yerr=Yerr,fmt='.',lw=0.5,c="b",ecolor="k") 
    ax.scatter(X,Y-clippedMeanMag,marker='o',s=10,c="b",label=label) 
    ax.xaxis.set_major_locator(month_locator) # Locator for major axis only.
    ax.xaxis.set_minor_locator(day_locator)
    ax.xaxis.set_major_formatter(year_month_formatter)
    ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
    ax.grid()
        
    if FLAG_PSF:  
        ax.set_title(f"Color {CBAND} psf Magnitude difference for pair {idx}")
          
    else:
        ax.set_title(f"Color {CBAND} Aperture Magnitude difference for pair {idx}")
               
                
    ax.set_xlabel("date obs") 
    ax.set_ylabel("dev (mmag)") 
    ax.legend(loc="upper left")
    ax.set_ylim(-60,60)
    plt.tight_layout()
    plt.show()
    
  
    
    

## Histograms

In [ ]:
thresholds = np.array([15.,16.,17.])
histoList1 = []
histoList2 = []
histoList3 = []
histoLists = [ histoList1, histoList2, histoList3]
Nthr = len(thresholds)
color=["b","g","r"]

In [ ]:
the_list_of_LC_average = avgpsfmag
the_list_of_LC_residuals = residuals
NLC = len(the_list_of_LC_average)

## Loop on threshold
for ith,th in enumerate(thresholds):
    for iobs in range(NLC):
        mag = the_list_of_LC_average[iobs]
        res = the_list_of_LC_residuals[iobs] 
        if mag < th:
            histoLists[ith].append(res)

In [ ]:
for ith,th in enumerate(thresholds):
    print(ith,th,len(histoLists[ith]))
    histoLists[ith] = np.hstack(histoLists[ith])
    print(ith,th,len(histoLists[ith]))
 


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,5))
for ith,th in enumerate(thresholds):
    data = histoLists[2-ith]
    sigma = np.std(data)
    m = thresholds[2-ith]
    label = f" m < {m:.1f} mag\n$\sigma$ = {sigma:.1f} mmag"
    ax.hist(data,bins=50,range=(-100,100),facecolor=color[2-ith],alpha=0.9,label=label)
ax.legend(loc="upper left")
if FLAG_PSF:
    ax.set_title(f"Color curve residuals in band {CBAND} (PSF Flux)")
else:
    ax.set_title(f"Color curve residuals in band {CBAND} (Aperture Flux)")

ax.set_xlabel("residuals (mmag)")